# Try Turi

In [148]:
from pymongo import MongoClient
import pandas as pd
from utils import preprocess_reviews, data_manager
import graphlab as gl
from utils import text_helpers

from matplotlib import pyplot as plt
import seaborn as sns
% matplotlib inline

%load_ext autoreload
%autoreload 2

from IPython.core.display import HTML
import urllib2
HTML(urllib2.urlopen('http://seanwade.com/jupyter.css').read())

In [10]:
client = MongoClient()
db = client.yelp

In [11]:
review_objects = data_manager.get_business_reviews('mVHrayjG3uZ_RLHkLj-AMg')
reviews = [x['text'] for x in review_objects]

In [14]:
sf = gl.SFrame(reviews)
sf

X1
This place wasDELICIOUS!! My parents ...
Can't miss stop for thebest Fish Sandwich in ...
This place should have alot more reviews - but ...
This place was very good.I found out about Emil's ...
Oldschool.....traditional ...
Seen this restaurant on25 best places in ...
Good fish sandwich.
Wonderful reuben. Mapshown on Yelp page is ...
"After a morning of ThriftStore hunting, a friend ..."
"This is a hidden gem, noreally. It took us ..."


In [21]:
bow = gl.text_analytics.count_words(sf['X1'])
sf['bow'] = bow

In [22]:
bow.dict_has_any_keys(['reuben'])

dtype: int
Rows: 25
[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]

In [26]:
sf['nouns'] = gl.text_analytics.extract_parts_of_speech(sf['X1'], chosen_pos=[gl.text_analytics.PartOfSpeech.NOUN])
sf

X1,bow,nouns
This place wasDELICIOUS!! My parents ...,"{'sandwich,': 1, 'they':2, 'sandwich.': 2, ...","{'NOUN': {'people': 1,'Things': 1, 'fish': 3, ..."
Can't miss stop for thebest Fish Sandwich in ...,"{'pittsburgh.': 1, 'for':1, 'fish': 1, 'stop': 1, ...",{'NOUN': {}}
This place should have alot more reviews - but ...,"{'all': 2, 'forget': 1,'just': 1, ""don't"": 3, ...","{'NOUN': {'prices': 1,'people': 1, 'taste': 2, ..."
This place was very good.I found out about Emil's ...,"{'love': 1, 'town,': 1,'people': 1, 'when': 1, ...","{'NOUN': {'town': 1,'sandwich': 1, 'shot' ..."
Oldschool.....traditional ...,"{'enjoy': 1, 'and': 3,'equally': 1, 'love': 1, ...","{'NOUN': {'vestige': 1,'school': 1, 'mom': 1, ..."
Seen this restaurant on25 best places in ...,"{'and': 1, 'this': 1,'all': 1, 'there': 2, ...","{'NOUN': {'sandwich': 1,'service': 1, 'places': ..."
Good fish sandwich.,"{'fish': 1, 'good': 1,'sandwich.': 1} ...","{'NOUN': {'fish': 1,'sandwich': 1}} ..."
Wonderful reuben. Mapshown on Yelp page is ...,"{'a': 2, 'on': 1,'different': 1, 'for' ...","{'NOUN': {'directions':1, 'call': 1, 'reuben': ..."
"After a morning of ThriftStore hunting, a friend ...","{'all': 1, 'huge!': 1,'go': 1, 'yinzer': 1, ...","{'NOUN': {'pound': 1,'neighborhood': 4, ..."
"This is a hidden gem, noreally. It took us ...","{'and': 1, 'the': 5,'right': 1, 'sandwich,': ...","{'NOUN': {'ribeye': 1,'what': 1, 'selection': ..."


In [27]:
sf['nouns'][3]

{'NOUN': {'Things': 1,
  'bar': 1,
  'beer': 1,
  'fish': 1,
  'food': 1,
  'lounge': 1,
  'part': 1,
  'people': 1,
  'place': 3,
  'places': 1,
  'restaurant': 1,
  'sandwich': 1,
  'shot': 1,
  'show': 1,
  'town': 1}}

## LDA

In [28]:
docs = sf['bow'].dict_trim_by_keys(gl.text_analytics.stopwords(), exclude=True)

In [32]:
model = gl.topic_model.create(sf['bow'], num_topics=50, num_iterations=50)

Learning a topic model

Number of documents        25

Vocabulary size       939

Running collapsed Gibbs sampling

+-----------+---------------+----------------+-----------------+

| Iteration | Elapsed Time  | Tokens/Second  | Est. Perplexity |

+-----------+---------------+----------------+-----------------+

| 10        | 13.855ms      | 1.30272e+06    | 0               |

+-----------+---------------+----------------+-----------------+

In [36]:
model.get_topics(output_type='topic_words')

words
"[food, huge, stuffed,italian, cabbage] ..."
"[chicken, &, good, da,waitress] ..."
"[week, visit, time,sandwich,, day] ..."
"[place, friendly, real,wonderful, good,] ..."
"[-, well., reuben,looked, thick] ..."
"[service, wife, home,quality, menu] ..."
"[back, place.,neighborhood, friend, ..."
"[fish, sandwich, don't,-, special] ..."
"[good, emil's, great,crispy, there.] ..."
"[i'm, great, it., find,fried] ..."


In [38]:
cursor = db.reviews.find()
review_list = []
for review in cursor:
    review_list.append(review['text'])

In [43]:
docs = gl.SFrame(review_list)
docs = gl.text_analytics.count_words(docs['X1'])
docs = docs.dict_trim_by_keys(gl.text_analytics.stopwords(), exclude=True)
docs = gl.text_analytics.trim_rare_words(docs)

In [67]:
model = gl.topic_model.create(docs)

CPU times: user 4 µs, sys: 10 µs, total: 14 µs
Wall time: 37 µs


Learning a topic model

Number of documents    628000

Vocabulary size    839527

Running collapsed Gibbs sampling

+-----------+---------------+----------------+-----------------+

| Iteration | Elapsed Time  | Tokens/Second  | Est. Perplexity |

+-----------+---------------+----------------+-----------------+

| 10        | 49.99s        | 7.50488e+06    | 0               |

+-----------+---------------+----------------+-----------------+

In [49]:
model.get_topics(output_type='topic_words')

words
"[service, day, don't,years, location] ..."
"[back, great, people,don't, i'm] ..."
"[told, time, i'm, i've,didn't] ..."
"[food, -, love, great,eat] ..."
"[place, good, great,time, restaurant] ..."
"[it's, good, &, pretty,nice] ..."
"[it's, car, time, asked,make] ..."
"[i've, didn't, wait, it.,give] ..."
"[ordered, good, order,night, service] ..."
"[place, staff, pizza,can't, nice] ..."


In [50]:
model.get_topics()

topic,word,score
0,service,0.0250971701014
0,day,0.0112586678505
0,don't,0.00953415385514
0,years,0.0089772996153
0,location,0.00890215319539
1,back,0.0213038095198
1,great,0.0151302688773
1,people,0.0144132928858
1,don't,0.01428310514
1,i'm,0.0135208464543


In [51]:
docs_in_topic_0 = docs[model.predict(docs) == 0]

In [59]:
pred = model.predict(docs, output_type='probability')
pred

dtype: array
Rows: 100000
[array('d', [0.12790697674418605, 0.10465116279069768, 0.08139534883720931, 0.10465116279069768, 0.11627906976744186, 0.05813953488372093, 0.10465116279069768, 0.09302325581395349, 0.12790697674418605, 0.08139534883720931]), array('d', [0.0851063829787234, 0.09929078014184398, 0.06382978723404255, 0.07092198581560284, 0.04964539007092199, 0.1347517730496454, 0.05673758865248227, 0.20567375886524822, 0.1347517730496454, 0.09929078014184398]), array('d', [0.09375, 0.078125, 0.078125, 0.078125, 0.109375, 0.109375, 0.140625, 0.125, 0.109375, 0.078125]), array('d', [0.08860759493670886, 0.06329113924050633, 0.06329113924050633, 0.10126582278481013, 0.1518987341772152, 0.10126582278481013, 0.08860759493670886, 0.10126582278481013, 0.1518987341772152, 0.08860759493670886]), array('d', [0.06493506493506493, 0.11688311688311688, 0.09090909090909091, 0.06493506493506493, 0.12987012987012986, 0.09090909090909091, 0.09090909090909091, 0.1038961038961039, 0.142857142857142

In [61]:
model['topics']

topic_probabilities,vocabulary
"[1.92683127971e-07,3.96223574245e-06, ...",alternative
"[0.00673446800572,1.88677892498e-07, ...",good
"[1.92683127971e-07,1.88677892498e-07, ...","subway,"
"[1.92683127971e-07,1.88677892498e-07, ...","overall,"
"[1.92683127971e-07,0.000477543745911, ...",fresh.
"[1.92683127971e-07,1.88677892498e-07, ...",year.
"[1.92683127971e-07,1.88677892498e-07, ...",cooked
"[0.0039058796871,1.88677892498e-07, ...",order
"[4.04634568739e-06,9.45276241413e-05, ...",mr
"[1.92683127971e-07,1.88677892498e-07, ...",voted


In [62]:
model.save('my_model')
new_model = graphlab.load_model('my_model')

new_model = graphlab.topic_model.create(docs,
                                        num_topics=m['num_topics'],
                                        initial_topics=m['topics'])

In [ ]:
associations = graphlab.SFrame({'word':['season', 'club', '2008', '2009'],
                                'topic': [1, 1, 2, 2]})

## Sentiment Analysis

In [108]:
review_objects = data_manager.get_business_reviews('4bEjOyTaDG24SY5TxsaUNQ')
reviews = [x['text'].encode('utf-8') for x in review_objects]
sf = gl.SFrame(reviews)

In [111]:
product_model = gl.product_sentiment.create(sf, features=['X1'])

Tokenizing...

TF-IDF transform...

Creating inverted index...

Creating query expansion model...

Saving data for querying...

In [121]:
product_model.get_most_positive??

In [122]:
a = product_model.get_most_positive(['Vegas'])['X1']

In [124]:
list(a)

['The day after a superb dinner at Guy Savoy (getting in later than my friends who were in Vegas to "party" and still waking up before them) I woke up early - feeling great I went for a run outside, the first time I\'d ever done so in Vegas - what they say about it being "a dry heat" is right and the mornings are dry and cool - running from the Venetian down to Mandalay Bay and back I was more dehydrated than I normally am on a run twice the distance.  Drinking plenty of water and showering I\'d worked up quite an appetite and made my way out for another walk south on the strip - this time to Mon Ami Gabi at Paris.  Arriving at 7:20am I was greeted by the friendly hostesses at the door and led promptly to a prime seat on the patio where I watched Vegas roll by - the best part about early morning Vegas is that there are only two sorts of people - the early morning joggers and the people just heading home.  I laughed like hell when a guy came up and yelled "high five - yellow shirt - I l

In [138]:
a = product_model.sentiment_summary??

In [ ]:
a = product_model.sentiment_summary

In [ ]:
a = product_model.sentiment_summary

In [146]:
a = product_model.sentiment_summary(keywords=['fish'])


In [139]:
pd.DataFrame(a.to_numpy(), columns=['keyword', 'sd_sentiment', 'mean_sentiment', 'review_count'])

TypeError: __init__() got an unexpected keyword argument 'reverse'

In [97]:
product_model.sentiment_summary(['service'])

keyword,sd_sentiment,mean_sentiment,review_count
service,0.436567860208,0.677864034014,10


In [104]:
product_model.review_searcher.query('Cr\xc3\xa8me Brulee', num_results=50)

score,__review_id,X1,sentiment_score
21.5002055173,4367,The quiche Lorraine islight and lovely.\n\nOur ...,0.98932230757
16.1333060022,4283,Everything was absolutelyamazing! From the ...,0.999997563257
15.551026925,3685,Came here for my birthdaydinner with a special ...,0.999999961179
12.494580323,3274,You must order steakfrites when you come ...,0.999999922323
11.8455604276,4732,I absolutely loved theview at Mon Ami Gabi. We ...,0.990928325544
11.7381686126,4952,I had reservations abouttrying this place due to ...,0.999969999151
10.6182049463,5083,first of all I shouldtell you this French ...,0.987676939541
10.3788331073,142,After searching Yelp forweeks and posting a ...,1.0
9.19591923992,5318,This is a must stop forme on every Vegas trip. ...,0.999950981081
9.14200929143,1719,Steak Frites wasincredible as usual. ...,0.999736179506


In [103]:
product_model.review_searcher.query??

In [149]:
pm = text_helpers.ProductModel('4bEjOyTaDG24SY5TxsaUNQ')

Tokenizing...

TF-IDF transform...

Creating inverted index...

Creating query expansion model...

Saving data for querying...

In [151]:
pm.get_most_negative()

'Unfortunately for Mon Ami Gabi, my rating is influenced largely by the way their staff handled a recent incident that occurred while I was eating there. \n\nWe arrived at Mon Ami Gabi after the 7:00 showing of the Jersey Boys. Our reservation was not until 9:30, but they were able to seat us immediately at 9:15. We ordered the Warm Brie Cheese, which the menu states is served with black pepper honey, hazelnuts, and croutons. While I still enjoyed the dish, there was actually only ONE hazelnut on the plate. \n\nFor our main courses, I ordered the French Onion Soup and my sister ordered the Grilled Salmon Salad. The soup was fine as far as French Onion Soup goes. My sister said the individual components of the salad were fine, but trying to eat all of the different parts together made for a strange taste in her opinion. \n\nWhen it came time to pay, I reached down next to my chair where my purse had been sitting during the entire meal to retrieve my wallet, only to find my purse was not